In [3]:
#import sys
#
#sys.path.append('/home/ondrej/soft/anaconda2/bin')

import numpy as np
import pandas as pd
import fiona
from mpl_toolkits.basemap import Basemap

ImportError: /usr/lib/liblwgeom-2.1.8.so: undefined symbol: GEOSDelaunayTriangulation

There are several good resources for map drawing, for instace:

The one from [sensitivecities](http://sensitivecities.com/so-youd-like-to-make-a-map-using-python-EN.html#.Vr2I43UrKkA "quite dense in info!").

In [3]:
shp = fiona.open('data/roads.shp')
shp.bounds

(-0.5177851, 51.2832439, 0.3385625, 51.6986909)

In [18]:
import sys

for path in sys.path:
    print(path)



/home/ondrej/soft/anaconda2/lib/python27.zip
/home/ondrej/soft/anaconda2/lib/python2.7
/home/ondrej/soft/anaconda2/lib/python2.7/plat-linux2
/home/ondrej/soft/anaconda2/lib/python2.7/lib-tk
/home/ondrej/soft/anaconda2/lib/python2.7/lib-old
/home/ondrej/soft/anaconda2/lib/python2.7/lib-dynload
/home/ondrej/soft/anaconda2/lib/python2.7/site-packages/Sphinx-1.3.1-py2.7.egg
/home/ondrej/soft/anaconda2/lib/python2.7/site-packages/setuptools-19.6.2-py2.7.egg
/home/ondrej/soft/anaconda2/lib/python2.7/site-packages/singledispatch-3.4.0.3-py2.7.egg
/home/ondrej/soft/anaconda2/lib/python2.7/site-packages
/home/ondrej/soft/anaconda2/lib/python2.7/site-packages/cryptography-1.0.2-py2.7-linux-x86_64.egg
/home/ondrej/soft/anaconda2/lib/python2.7/site-packages/IPython/extensions
/home/ondrej/.ipython
/home/ondrej/soft/anaconda2/bin
/home/ondrej/soft/anaconda2/bin
